In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
#  Import the Wikipedia JSON file
# ./Weather_Database/WeatherPy_Database.csv
file_dir = './Resources/'
f'{file_dir}wikipedia-movies.json'


'./Resources/wikipedia-movies.json'

In [3]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file) #now wiki_... a list of dictionary

FileNotFoundError: [Errno 2] No such file or directory: './Resources//wikipedia-movies.json'

In [4]:
#Check how many records were pulled in
len(wiki_movies_raw)

NameError: name 'wiki_movies_raw' is not defined

In [5]:
# See the first 5 records
wiki_movies_raw[:5]

NameError: name 'wiki_movies_raw' is not defined

In [6]:
# Some records in the middle
wiki_movies_raw[3600:3605]

NameError: name 'wiki_movies_raw' is not defined

In [7]:
# Kaggle data is already in flat-file formats, we'll just pull them into Pandas DataFrames 
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')
ratings.sample(n=5)

FileNotFoundError: [Errno 2] No such file or directory: './Resources/movies_metadata.csv'